# Fairness-Aware Feature Selection

## COMPAS Dataset

### 1) Preprocess data

In [1]:
#Load data 
import itertools

import numpy as np
import pandas as pd
from scipy import optimize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.svm import SVC
from sklearn.utils import shuffle
import copy
import math

fp = '../data/compas-scores-two-years.csv'
compas_df = pd.read_csv(fp)

In [2]:
def process_compas_dataset(compas_df): 
    #Drop Missing values and subset on columns needed
    compas_df.dropna()
    compas_subset = compas_df[["sex","age","age_cat","race","priors_count","c_charge_degree","c_jail_in", "c_jail_out",'two_year_recid']]
    compas_subset["two_year_recid"] = compas_subset["two_year_recid"].apply(lambda x: -1 if x==0 else 1)
    
    #Only select Caucasian/African American, encode to 0/1
    compas_subset = compas_subset[(compas_subset["race"]=='Caucasian') |(compas_subset["race"]=='African-American') ]
    compas_subset["race_cat"] = compas_subset["race"].apply(lambda x: 1 if x == "Caucasian" else 0)
    compas_subset = compas_subset.drop(columns = "race")
    
    #Encode gender to 0/1
    compas_subset["gender_cat"] = compas_subset["sex"].apply(lambda x: 1 if x == "Female" else 0)
    compas_subset = compas_subset.drop(columns = "sex")
    
    #Encode charge degree to 0/1
    compas_subset["charge_cat"] = compas_subset["c_charge_degree"].apply(lambda x: 1 if x == "F" else 0)
    compas_subset = compas_subset.drop(columns = "c_charge_degree")
    
    #Calculate length of stay from jail out - jail in 
    compas_subset["length_stay"] = pd.to_datetime(compas_subset["c_jail_out"]) - pd.to_datetime(compas_subset['c_jail_in'])
    compas_subset["length_stay"] = compas_subset["length_stay"].apply(lambda x: x.days)
    compas_subset = compas_subset.drop(columns = ["c_jail_in","c_jail_out"])
    compas_subset['length_stay'] = compas_subset["length_stay"].apply(lambda x: 0 if x <= 7 else x)
    compas_subset['length_stay'] = compas_subset["length_stay"].apply(lambda x: 1 if 7< x <= 90 else x)
    compas_subset['length_stay'] = compas_subset["length_stay"].apply(lambda x: 2 if x > 90 else x)
    
    #Categorize priors count into 3 categories 
    compas_subset["priors_count"] = compas_subset["priors_count"].apply(lambda x: 0 if x==0 else x)
    compas_subset["priors_count"] = compas_subset["priors_count"].apply(lambda x: 1 if (1<=x<=3) else x)
    compas_subset["priors_count"] = compas_subset["priors_count"].apply(lambda x: 2 if x>3 else x)
    
    # Include age as categorical variable 
    compas_subset = compas_subset.drop(columns = ["age_cat"])
    
    compas_subset = compas_subset.dropna()
    y_label = compas_subset["two_year_recid"]
    protected_attribute = compas_subset["race_cat"]
    df = compas_subset.drop(columns=["two_year_recid","race_cat"])

    y_label, protected_attr, df = shuffle(y_label, protected_attribute, df, random_state = 0)

    return y_label.to_numpy(), protected_attr.to_numpy(), df.to_numpy()

In [3]:
y_label, protected_attr, X =  process_compas_dataset(compas_df)

train_index = int(len(X)*6./7.)
x_train, y_train, race_train = X[:train_index], y_label[:train_index], protected_attr[:train_index]
x_test, y_test, race_test = X[train_index:], y_label[train_index:],protected_attr[train_index:]

/Users/Nikhil/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## 2) Feature Selection

In this section, we implement the routines described in "Information Theoretic Measures for Fairness-aware Feature Selection." On the training data, we calculate Shapley coefficients for each of our features capturing effects on both accuracy and discrimation on our protected group (i.e. race).

In [37]:
"""This cell contains utility functions called in the proceeding cells."""

def get_uniq_vals_in_arr(arr):
    """Returns unique values in array.
    
    :param arr (np.array) n * m matrix
    :return (list) uniq_vals[i] contains unique values of ith column in arr
    """
    uniq_vals = []
    for id_col in range(arr.shape[1]):
        uniq_vals.append(np.unique(arr[:, id_col]).tolist())
    return uniq_vals


def powerset(seq):
    """
    Returns all the subsets of this set. This is a generator.
    """
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

In [38]:
"""This cell contains code for all the routines needed to calculate the Shapley coefficients."""

def get_info_coef(left, right):
    # Both arrays NEED same number of rows
    assert left.shape[0] == right.shape[0]
    num_rows = left.shape[0]
    num_left_cols = left.shape[1]
        
    concat_mat = np.concatenate((left, right), axis=1)
    concat_uniq_vals = get_uniq_vals_in_arr(concat_mat)
    concat_combos = list(itertools.product(*concat_uniq_vals))
    p_sum = 0
    for vec in concat_combos:
        p_r1_r2 = len(np.where((concat_mat == vec).all(axis=1))[0]) / num_rows
        p_r1 = len(np.where((left == vec[:num_left_cols]).all(axis=1))[0]) / num_rows
        p_r2 = len(np.where((right == vec[num_left_cols:]).all(axis=1))[0]) / num_rows
        
        if p_r1_r2 == 0 or p_r1 == 0 or p_r2 == 0:
            p_iter = 0
        else:
            p_iter = p_r1_r2 * np.log(p_r1_r2 / p_r1) / p_r1
        p_sum += np.abs(p_iter)
    return p_sum


def get_conditional_info_coef(left, right, conditional): 
    assert (left.shape[0] == right.shape[0]) and (left.shape[0] == conditional.shape[0])
    num_rows = left.shape[0]
    num_left_cols = left.shape[1]
    num_right_cols = right.shape[1]

    right_concat_mat = np.concatenate((right, conditional), axis=1)    
    concat_mat = np.concatenate((left, right_concat_mat), axis=1)
    concat_uniq_vals = get_uniq_vals_in_arr(concat_mat)
    concat_combos = list(itertools.product(*concat_uniq_vals))
    p_sum = 0
    for vec in concat_combos:
        p_r1_r2 = len(np.where((concat_mat == vec).all(axis=1))[0]) / num_rows
        p_r1 = len(np.where((left == vec[:num_left_cols]).all(axis=1))[0]) / num_rows
        p_r2 = len(np.where((concat_mat[:, num_left_cols: -num_right_cols] == vec[num_left_cols: -num_right_cols]).all(axis=1))[0]) / num_rows
        
        try:
            p_r1_given_r3 = len(np.where((concat_mat[:, :num_left_cols] == vec[:num_left_cols]).all(axis=1) & (concat_mat[:, -num_right_cols:] == vec[-num_right_cols:]).all(axis=1))[0]) / len(np.where((concat_mat[:, -num_right_cols:] == vec[-num_right_cols:]).all(axis=1))[0])
        except ZeroDivisionError:
            p_r1_given_r3 = 0
        
        if p_r1_r2 == 0 or p_r1 == 0 or p_r2 == 0 or p_r1_given_r3 == 0:
            p_iter = 0
        else:
            p_iter = p_r1_r2 * np.log(p_r1_r2 / p_r2) / p_r1_given_r3
        p_sum += np.abs(p_iter)
    return p_sum


def get_acc_coef(y, x_s, x_s_c, protected_attr):
    conditional = np.concatenate((x_s_c, protected_attr), axis=1)
    return get_conditional_info_coef(y, x_s, conditional)


def get_disc_coef(y, x_s, protected_attr):
    x_s_a = np.concatenate((x_s, protected_attr), axis=1)
    return get_info_coef(y, x_s_a) * get_info_coef(x_s, protected_attr) * get_conditional_info_coef(x_s, protected_attr, y)


def get_shapley_acc_i(y, x, protected_attr, i):
    """Returns Shapley coeffecient of ith feature in x."""
    
    num_features = x.shape[1]
    lst_idx = list(range(num_features))
    lst_idx.pop(i)
    power_set = [x for x in powerset(lst_idx) if len(x) > 0]
    
    shapley = 0
    for set_idx in power_set:
        coef = math.factorial(len(set_idx)) * math.factorial(num_features - len(set_idx) - 1) / math.factorial(num_features)
        
        # Calculate v(T U {i})
        idx_xs_incl = copy.copy(set_idx)
        idx_xs_incl.append(i)
        idx_xsc_incl = list(set(list(range(num_features))).difference(set(idx_xs_incl)))
        acc_incl = get_acc_coef(y.reshape(-1, 1), x[:, idx_xs_incl], x[:, idx_xsc_incl], protected_attr.reshape(-1, 1))
        
        # Calculate v(T)
        idx_xsc_excl = list(range(num_features))
        idx_xsc_excl.pop(i)
        idx_xsc_excl = list(set(idx_xsc_excl).difference(set(set_idx)))
        acc_excl = get_acc_coef(y.reshape(-1, 1), x[:, set_idx], x[:, idx_xsc_excl], protected_attr.reshape(-1, 1))
        
        marginal = acc_incl - acc_excl
        shapley = shapley + coef * marginal
    return shapley


def get_shapley_disc_i(y, x, protected_attr, i):
    """Returns Shapley coeffecient of ith feature in x."""
    
    num_features = x.shape[1]
    lst_idx = list(range(num_features))
    lst_idx.pop(i)
    power_set = [x for x in powerset(lst_idx) if len(x) > 0]
    
    shapley = 0
    for set_idx in power_set:
        coef = math.factorial(len(set_idx)) * math.factorial(num_features - len(set_idx) - 1) / math.factorial(num_features)
        
        # Calculate v_D(T U {i})
        idx_xs_incl = copy.copy(set_idx)
        idx_xs_incl.append(i)
        disc_incl = get_disc_coef(y.reshape(-1, 1), x[:, idx_xs_incl], protected_attr.reshape(-1, 1))
        
        # Calculate v_D(T)
        disc_excl = get_disc_coef(y.reshape(-1, 1), x[:, set_idx], protected_attr.reshape(-1, 1))
        
        marginal = disc_incl - disc_excl
        shapley = shapley + coef * marginal
    return shapley

In [39]:
# Calculate Shapley disc, acc coefs for each feature over training data
shap_acc = []
shap_disc = []
for i in range(5):
    acc_i = get_shapley_acc_i(y_train, x_train, race_train, i)
    disc_i = get_shapley_disc_i(y_train, x_train, race_train, i)
    
    shap_acc.append(acc_i)
    shap_disc.append(disc_i)

# Build Shapley output
feature_names = ["Prior Count", "Gender", "Charge Degree", "Length of Stay", "Age (Categorical)"]
shapley_df = pd.DataFrame(list(zip(feature_names, shap_acc, shap_disc)),
                          columns=["Feature", "Shapley (Accuracy)", "Shapley (Discrimination)"])
shapley_df = shapley_df.sort_values(by=["Shapley (Discrimination)"], ascending=[False]).reset_index(0, True)
shapley_df.to_csv("../output/compas-data-shapley-table.csv")

KeyboardInterrupt: 

In [41]:
pd.set_option('display.float_format', lambda x: '%.2E' % x)
shapley_df

,Feature,Shapley (Accuracy),Shapley (Discrimination)
0,Prior Count,2.46E+00,7.17E+06
1,Gender,1.37E+00,4.92E+06
2,Age (Categorical),1.29E+00,4.03E+06
3,Length of Stay,1.31E+00,3.68E+06
4,Charge Degree,1.24E+00,3.07E+06


We see that 'Prior Count' has the sharpest affect on both discrimination and accuracy, so eliminating it can prove problematic for a classifier. However, a feature such as 'Age (Categorical)' is relatively discriminatory but eliminating it would not seriously reduce accuracy from our results.

### 3) Classification

Now, we build an SVM model that predicts whether a user will or will not recidivate given the aforementioned features. We calculate accuracy as well as calibration, the difference between accuracy amongst the groups. Finally, we build submodels which eliminate each feature iteratively and calculate both metrics. 

In [60]:
test_acc = []
test_cal = []

# Build model for overall data inclusive of all features
svm = SVC(kernel="linear").fit(x_train, y_train)
idx_race_1, idx_race_0  = np.where(race_test == 1)[0], np.where(race_test == 0)[0]
test_acc.append(svm.score(x_test, y_test))
test_cal.append(svm.score(x_test[idx_race_1], y_test[idx_race_1]) - svm.score(x_test[idx_race_0], y_test[idx_race_0]))

# Eliminate one feature at a time build model
for id_feature in range(x_train.shape[1]):
    idxs = list(range(x_train.shape[1]))
    idxs.pop(id_feature)
    x_train_mod = x_train[:, idxs]
    x_test_mod = x_test[:, idxs]
    
    svm = SVC(kernel="linear").fit(x_train_mod, y_train)
    acc = svm.score(x_test_mod, y_test)
    cal = svm.score(x_test_mod[idx_race_1], y_test[idx_race_1]) - svm.score(x_test_mod[idx_race_0], y_test[idx_race_0])
    
    test_acc.append(acc)
    test_cal.append(cal)
    

index_names = ["None", "Prior Count", "Gender", "Charge Degree", "Length of Stay", "Age (Categorical)"]
test_acc = [x * 100 for x in test_acc]
test_cal = [x * 100 for x in test_cal]
results = pd.DataFrame(list(zip(index_names, test_acc, test_cal)),
                          columns=["Eliminating Feature", "Accuracy (%)", "Calibration (%)"])
results["Delta (%)"] = 2.00 - results["Calibration (%)"]
results.to_csv("../output/compas-data-test-results.csv")

In [67]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
results

,Eliminating Feature,Accuracy,Calibration (1-0),Delta (%)
0,None,68.17,2.00,-0.00
1,Prior Count,64.02,0.76,1.24
2,Gender,61.07,-0.06,2.06
3,Charge Degree,67.81,3.58,-1.58
4,Length of Stay,67.81,1.63,0.37
5,Age (Categorical),66.39,1.61,0.39


When evaluating the results, we must compare against the baseline calibration score of 2%.

Clearly, our results differ somewhat from our pretraining procedures. For example, we show that eliminating 'Prior Count' should result in the strongest drop in discrimation but the results suggest that actually eliminating 'Gender' yields the greatest benefit to discrimation. However, our FFS process showed that dropping 'Charge Degree' is unnecessary and the results prove that as removing it from the training set results in a significantly more discriminatory classifier.